In [3]:
import pandas as pd
import requests as r
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [69]:
def get_historical_data(latitude=53.55,
                      longitude=9.99,
                      variables='temperature_2m',
                      timezone='auto',
                      model='best_match',
                      start_date='1991-01-01',
                      end_date='2020-12-31'):
    payload = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": variables,
        "timezone": timezone,
        "models": model,
        "start_date": start_date,
        "end_date": end_date
    }


    resp = r.get("https://archive-api.open-meteo.com/v1/archive",
                 params=payload)
    data = pd.DataFrame.from_dict(resp.json()['hourly'])
    data['time'] = pd.to_datetime(
        data['time'], format='%Y-%m-%dT%H:%M')

    data = data.dropna()

    return data


def compute_climatology(latitude=53.55,
                      longitude=9.99,
                      variables='temperature_2m',
                      timezone='auto',
                      model='best_match',
                      start_date='1991-01-01',
                      end_date='2020-12-31',
                      year=2020):
    
    data = get_historical_data(latitude, longitude, variables,
                               timezone, model, start_date, end_date)
    data.resample('3H', on='time').first()
    data['doy'] = data.time.dt.strftime("%m%d")
    mean = data.groupby([data.doy, data.time.dt.hour]).mean(numeric_only=True).rename_axis(['doy','hour']).reset_index()
    # Create a fake date just in case
    mean['dummy_date'] = pd.to_datetime(mean['doy'] + str(year) + "T" + mean['hour'].astype(str).str.zfill(2) + "00", format='%m%d%YT%H%M')
    
    return mean

def get_ensemble_data(latitude=53.55,
                      longitude=9.99,
                      variables='temperature_2m,cloudcover,rain,snowfall,precipitation',
                      timezone='auto',
                      model='icon_seamless',
                      from_now=True):
    """
    Get the ensemble data
    """
    if model == 'icon_seamless':
        forecast_days = 7
    else:
        forecast_days = 14
    payload = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": variables,
        "timezone": timezone,
        "models": model,
        "forecast_days": forecast_days
    }

    resp = r.get("https://ensemble-api.open-meteo.com/v1/ensemble",
                 params=payload)
    data = pd.DataFrame.from_dict(resp.json()['hourly'])
    data['time'] = pd.to_datetime(
        data['time']).dt.tz_localize(resp.json()['timezone'])

    data = data.dropna()
    # Optionally subset data to start only from previous hour
    if from_now:
        data = data[data.time >= pd.to_datetime('now', utc=True).tz_convert(resp.json()['timezone']).floor('H')]

    return data

In [72]:
ens_forecast = get_ensemble_data(variables='temperature_2m')
clima = compute_climatology(variables='temperature_2m')

In [ ]:
var = 'temperature_2m'

traces = []
for col in ens_forecast.columns[ens_forecast.columns.str.contains(var)]:
    # First do the first 48 hrs
    traces.append(
        go.Scatter(
            x=ens_forecast.loc[ens_forecast.time <= ens_forecast.time.iloc[0] + pd.to_timedelta('48H'),'time'],
            y=ens_forecast.loc[ens_forecast.time <= ens_forecast.time.iloc[0] + pd.to_timedelta('48H'), col],
            mode='lines',
            name=col,
            showlegend=False),
    )
# Then the remaining as boxplot
traces.append(
    go.Box(
    y=ens_forecast.loc[ens_forecast.time > ens_forecast.time.iloc[0] + pd.to_timedelta('48H'), ens_forecast.columns.str.contains(var)],
    x=ens_forecast.loc[ens_forecast.time > ens_forecast.time.iloc[0] + pd.to_timedelta('48H'), 'time'],
    name=col,
))


go.Figure(traces)

In [ ]:
go.Figure(go.Box(
    y=ens_forecast.loc[ens_forecast.time > ens_forecast.time.iloc[0] + pd.to_timedelta('48H'), ens_forecast.columns.str.contains(var)].values.T,
    x=ens_forecast.loc[ens_forecast.time > ens_forecast.time.iloc[0] + pd.to_timedelta('48H'), 'time'].values,
    name=col,
))